In [1]:
import pandas as pd
import numpy as np
import os
from final_models import HGAT, NodeAttentionLayer
import torch
from torch import nn
from tqdm import tqdm
from scipy import sparse as sp
from numpy import dot
from numpy.linalg import norm
from dataset import recipe_dataset
import random
from utils import recall, custom_train_valid_split
import torch.optim as optim
import torch.nn.functional as F

path = '/opt/ml/recipe/recipe_data'

# recipe = pd.read_csv(os.path.join(path, '레시피_메타정보_raw.csv'))

# recipe_seq = pd.read_csv(os.path.join(path, '레시피_순서_내용_raw.csv'))

# recipe_user = pd.read_csv(os.path.join(path, '레시피_유저_내용_raw.csv'))
recipe_ingredient = pd.read_csv(os.path.join(path, '레시피_재료_내용_raw.csv'))
recipe_ingredient.dropna(subset=['재료_아이디'],inplace=True)
# recipe_cat = pd.read_csv(os.path.join(path,'레시피_분류_내용_raw.csv'))
# recipe_new_cat = pd.read_csv(os.path.join(path,'레시피_카테고리_정제.csv'))

# recipe_tag = pd.read_csv(os.path.join(path, '레시피_태그_내용_raw.csv'))
# user_data = pd.read_csv(os.path.join(path, '유저_메타정보_raw.csv'))
# ingredients = pd.read_csv(os.path.join(path, '재료_메타정보_raw.csv'))

# ingredient_meta = pd.read_csv(os.path.join(path,'재료_메타정보_1.0.csv'))
# ingredient_keyword = pd.read_csv(os.path.join(path,'재료_키워드_내용_raw.csv'))
# ingredient_recipe = pd.read_csv(os.path.join(path,'재료_레시피_내용_raw.csv'))

# # 피쳐를 dense 하게 학습 하는 것과 sparse 하게 학습하는 것과 결과 차이가 있는가?
# user_node, recipe_node, ing_node = dataset.get_node()

# user_realation_matrix, recipe_relation_matrix = dataset.get_matrix()

In [2]:
def recall_k(prediction,labels, topk):
    recall_at_k = 0
    for i in range(prediction.shape[0]):
        r_i = prediction[i].topk(topk).indices.tolist()
        l_i = torch.where(labels[i] == 1)[0].tolist()
        recall_i = len(set(r_i) & set(l_i)) / min(topk,len(l_i))
        recall_at_k += recall_i
    return recall_at_k/prediction.shape[0]

In [3]:
train = pd.read_csv(os.path.join(path, 'train셋(73609개-220603_192931).csv'))
test = pd.read_csv(os.path.join(path, 'test셋(4422개-220603_192931).csv'))

In [4]:
dataset = recipe_dataset( train,test, recipe_ingredient )

사용할 유저 수 : 4422
사용할 레시피 수 : 5900
사용할 재료 수: 911


In [5]:
users, recipes, ings = dataset.get_user_recipe_ing()

In [6]:
user_rel_matrix, recipe_rel_matrix = dataset.get_matrix()

In [7]:

device = 'cuda'
hidden_dim = 32
seed = 9

In [8]:
# node = NodeAttentionLayer(32, 512, 0.5 ,32,'cuda')
# e = node(model.user_embedding(torch.tensor([0])).to('cuda') , model.recipe_embedding[torch.tensor(user_rel_matrix[0][0])].to('cuda'))

In [9]:
model = HGAT(
            n_user = len(users),
            n_recipe = len(recipes), 
            n_ing = len(ings),
            u_dim = hidden_dim, 
            r_dim = 512, 
            i_dim = 512, 
            out_features = hidden_dim,            
            drop_out = 0.5, 
            n_heads=3,
            user_relation_matrix = user_rel_matrix, 
            recipe_relation_matrix = recipe_rel_matrix,            
            device = device
            )

In [10]:
user2idx = dataset.user2idx
recipe2idx = dataset.recipe2idx

label = torch.zeros(len(users), len(recipes))
urm = torch.zeros(len(users), len(recipes))
for u in tqdm(train.유저_아이디.unique()):
    u_idx = user2idx[u]
    for r in test[test.유저_아이디==u].레시피_아이디.unique():
        r_idx = recipe2idx[r]
        label[u_idx][r_idx] = 1
    for r in train[train.유저_아이디==u].레시피_아이디.unique():
        r_idx = recipe2idx[r]
        urm[u_idx][r_idx] = 1

100%|██████████| 4422/4422 [00:27<00:00, 159.93it/s]


In [11]:
epochs = 200
topk = 100
optimizer = optim.Adam(model.parameters(), 
                       lr=1e-2,
                       weight_decay=1e-3)
criterion = nn.CrossEntropyLoss()

In [12]:
torch.autograd.set_detect_anomaly(True)

In [13]:
best_loss = 1e9
topk = 100

epochs = 100
best_epochs = 0
patient = 50
count = 0
urm = urm.reshape(4422,1,5900)

for e in tqdm(range(epochs)):
    model.train()
    train_pred = model(
                        list(range(len(users))),
                        []
                    )

    preds = train_pred.to('cpu')    
    loss_train = criterion(preds, urm)

    print('---------------------------------------------------------------------')
    if loss_train.detach().item() < best_loss:
        # torch.save(model.state_dict(), '/opt/ml/recipe/HGAT/model_save/best_model.pt')
        print('save best model!')
        best_loss = loss_train.detach().item()
        best_epochs = e
        preds[torch.where(urm)] = 0
        print(f"recall@{topk} :", recall_k(preds.squeeze(1),label,topk))
        count = 0
    else:
        count += 1
        if count == patient:
            print(f'patient == {patient}')
            break

    optimizer.zero_grad()
    loss_train.backward()
    optimizer.step()
    print(f'epochs : {e}')
    print('train loss :',loss_train)    

  0%|          | 0/100 [00:00<?, ?it/s]

H_U : -997.2159016132355
